## algorithm design and anlysis-2025 spring  homework 4
**Deadline**：2025.5.14

**name**:


note：
---
1. 带有\*的题目，申请免上课的同学，必须完成，其他同学选作；
2. 请独立完成，如求助了他人或者大模型，请著明，并且不可省略算法分析部分；
4. 如若作答有雷同，全部取消成绩；
3. 需要书面作答的题目，可以通过引用图片的形式添加，但是注意上传项目时包含所引用的图片的源文件；
4. $log_n$ 默认表示$log_2{n}$;

## 问题 1 
**最小生成树（Minimum Spanning Tree）**

设  **G**  为一个带权重的连通无向图，且所有边的权重均不相等。令$e_i$ 为权重第 $i$ 小的边。最小生成树（MST）是否必须包含 $e_1$ ? 同理，是否必须包含 $e_2$ 和 $e_3$ ? 若必须包含，请给出证明；否则，请构造反例。需从基本原理论证，不能依赖割引理(cut lemma) 或 Prim/Kruskal算法的正确性。


answer:把所有边按权重从小到大排好，最轻的那条边 e1肯定要选，否则把它补进来会绕出一个只包含比它更重的边的环，随便去掉环中任意一条更重的边就能得到更轻的生成树，跟“最小”矛盾；第二轻的e2也一样，跳过它的话也能用它去替换环里某条更重的边；但第三轻的e3就不必选了——比如一个三角形，边权 1、2、3，最小生成树只要 1 和 2，权重为 3 的那条完全可以不加。

## 问题 2 
**瓶颈生成树（Bottleneck Spanning Tree）**

带有权重的无向图 $G(V,E,w)$ 的瓶颈生成树，表现为：在所有生成树中，最大权重边的权重值最小。即，BST $T$ 最小化瓶颈损失 $c(T)=max_{e \in T}{w(e)}$。

1. 证明 $G$ 的每一个最小生成树（MST）都是瓶颈生成树（BST）
2. 设计一个线性时间复杂度的算法：， 对于一个图 $G(V,E,w)$ 和一个整数 $b$，判断图 $ G$ 是否存在一个瓶颈生成树，其最大权重边的权重不超过 $b$，分析算法设计思路，并基于python编程实现。
3. 设计一个线性时间复杂度的算法：对于给定的图 $G(V,E,w)$，找到其瓶颈生成树，分析算法设计思路，并基于python编程实现。

idea：先假设T是一棵最小生成树，它的所有边权之和比任何其它生成树都要小。现在如果存在另一棵生成树F，使得F的最大边权比T的最大边权还要小，那么F的每一条边权就都不超过T的最大边权。这样一来，F里所有边权加起来必然不会大于T的最大边权乘以边数，自然也不会超过T的总权重——甚至可能更小。可这又跟T“总权重最小”的定义相矛盾。因此，不可能有任何一棵生成树的最大边权小于T的最大边权，也就是说，作为最小生成树的T同时也把“最大边权”做到了最小，换句话说它也是一棵瓶颈最小生成树。

In [2]:
from collections import deque

def exists_bst(G: dict[int, list[tuple[int,int]]], b: int) -> bool:

    if not G:
        return True
    start = next(iter(G))
    seen = {start}
    q = deque([start])
    while q:
        u = q.popleft()
        for v, w in G[u]:
            if w <= b and v not in seen:
                seen.add(v)
                q.append(v)
    return len(seen) == len(G)


def find_bst(G: dict[int, list[tuple[int,int]]]) -> list[tuple[int,int,int]]:
    weights = sorted({w for u in G for v, w in G[u]})
    lo, hi = 0, len(weights) - 1
    while lo < hi:
        mid = (lo + hi) // 2
        if exists_bst(G, weights[mid]):
            hi = mid
        else:
            lo = mid + 1
    b_opt = weights[lo]

    start = next(iter(G))
    seen = {start}
    stack = [start]
    bst_edges = []
    while stack:
        u = stack.pop()
        for v, w in G[u]:
            if w <= b_opt and v not in seen:
                seen.add(v)
                bst_edges.append((u, v, w))
                stack.append(v)
    return bst_edges


G = {
        'A': [('B',1), ('C',3)],
        'B': [('A',1), ('C',2)],
        'C': [('A',3), ('B',2)],
}

print(exists_bst(G, 1))  
print(exists_bst(G, 2))  
bst = find_bst(G)
print(bst)
 

False
True
[('A', 'B', 1), ('B', 'C', 2)]


## 问题 3

**道路网（Road Network）**

假设有一个以图 $ G(V, E, l) $ 表示的道路网络，连接了一组城市 $ V $。我们假设该网络是有向的，并且每条道路 $(u, v) \in E$ 都有一个非负的长度 $ l(u, v) $。一条新的道路即将被建造，因此有一个列表 $ E' $ 包含它可以连接的城市对。每对 $(u, v) \in E'$ 都有一个对应的长度 $ l'(u, v) $。我们希望选择一对城市，使得两个城市 $ s, t \in V $ 之间的距离减少最大。请为此问题编写一个高效的算法，并详细解释算法的正确性和复杂度。


In [3]:
import heapq
from collections import defaultdict

def best_new_road(V: int,
                  edges: list[tuple[int,int,float]],
                  candidates: list[tuple[int,int,float]]
                 ) -> tuple[tuple[int,int], float]:
 
    G = [[] for _ in range(V)]
    for u, v, w in edges:
        G[u].append((v, w))
    by_source = defaultdict(list)
    for u, v, new_w in candidates:
        by_source[u].append((v, new_w))

    best_delta = 0.0
    best_edge = None

    for u, cand_list in by_source.items():
        dist = [float('inf')] * V
        dist[u] = 0.0
        hq = [(0.0, u)]
        while hq:
            d, x = heapq.heappop(hq)
            if d > dist[x]:
                continue
            for y, w in G[x]:
                nd = d + w
                if nd < dist[y]:
                    dist[y] = nd
                    heapq.heappush(hq, (nd, y))

        for v, new_w in cand_list:
            if dist[v] < float('inf'):
                delta = dist[v] - new_w
                if delta > best_delta:
                    best_delta = delta
                    best_edge = (u, v)

    return best_edge, best_delta


V = 3
edges = [(0,1,10), (1,2,5), (0,2,20)]
cands = [(0,2,12), (1,2,3), (0,1,6)]
best, gain = best_new_road(V, edges, cands)
print("最佳新路 =", best)     
print("最大距离减少 =", gain)  

最佳新路 = (0, 1)
最大距离减少 = 4.0


## 问题 4

**逃离问题**

一个 $ n \times n $ 的网格是一个无向图，由 $ n $ 行和 $ n $ 列的顶点组成，如下图所示。我们用 $(i,j)$ 表示第 $ i $ 行和第 $ j $ 列的顶点。除了边界顶点，网格中的所有顶点都有四个邻居，即满足 $ i = 1, i = n, j = 1 $ 或 $ j = n $ 的点 $(i,j)$。

给定网格中的 $ m \leq n^2 $ 个起点 $(x_1, y_1), (x_2, y_2), \cdots , (x_m, y_m)$，逃离问题是确定是否存在 $ m $ 条顶点不相交的路径（即路径之间不相交），从这些起点到边界上的任意 $ m $ 个不同点。例如，图1中的网格存在逃离。

(1) 该问题可以看作是一个最大流问题。考虑一个流网络，其中顶点和边都有容量。也就是说，进入任何给定顶点的总正流量受到容量限制。证明在具有边和顶点容量的网络中确定最大流可以简化为在具有可比大小的普通流网络上的最大流问题。更准确地说，你需要将一个具有顶点和边容量的网络 $ G = (V,E) $ 转换为另一个仅具有边容量的网络 $ G' = (V', E') $，使得两个网络上的最大流相同，并且你构建的新网络具有 $ V' = O(V) $ 个顶点和 $ E' = O(E) $ 条边。你可以假设网络是连通的。

(2) 描述一个解决逃离问题的高效算法，并分析其运行时间。


<div align="center"> <img alt="图片" src="./fig/escepe-p.png"> </div>
<center> 图2. 逃脱问题网格，起始顶点为黑色，其他网格顶点为白色</center>

idea：
1.对于第一个子问题，我们需要把“每个格点最多只允许一条路径经过”这样的顶点容量限制，转换成普通网络中的边容量限制。具体做法是先在原网格上增设超级源s和超级汇t，分别将s通过容量为1的有向边连向每个出发点，将所有边界格点通过容量为1的有向边连向t。接着对原网格内的每个顶点v执行“拆分”：在v_in→v_out之间加一条容量为1的有向边，原本所有指向v的边都改为指向v_in，所有自v出发的边都从v_out出发。这样一来，对顶点的访问次数限制就变成了对这条内部边的容量限制，网络中只剩下边容量约束。最终我们得到一个只有边容量的有向流网络，并且它的最大流值等价于原问题中能找到的顶点不相交路径条数。

2.高效算法，只需在这个新构造的流网络上运行一次最大流即可判断逃离是否可行。该网络的顶点数和边数均为O(n²)，而对于单位容量网络，Dinic 算法能在O(√V·E) ≈ O(n³)的时间内算出从s到t的最大流值。如果最大流至少为m，就表示存在m条顶点不相交的路径；否则就不可能完成全部起点的逃离。这样，我们便在多项式时间内解决了带顶点容量约束的逃离问题